# PIPELINE FINALE

## Initialisation de la pipeline

### 0.1 - Import des librairies nécéssaires au code

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
import shutil

from concurrent.futures import ThreadPoolExecutor, as_completed
from PIL import Image, UnidentifiedImageError
from tensorflow import keras
from tensorflow.keras.models import load_model
# serializable
from tensorflow.keras import backend as K

### 0.2 - Définition des constantes

In [2]:
# -------------------------------- General Parameters
SEED = 42
# -------------------------------- Images Parameters
IMAGE_H = 128
IMAGE_W = 128
BATCH_S = 16
# -------------------------------- Classes indexes
PAINTING_IDX = 0
PHOTO_IDX = 1
SCHEMA_IDX = 2
SKETCH_IDX = 3
TEXT_IDX = 4
# -------------------------------- Folders
DATASET_DIRECTORY = "dataset_livrable_1/"
PHOTOS_DIRECTORY = "final_pipeline/photos"
DENOISED_PHOTOS_DIRECTORY = "final_pipeline/denoised_photos"
MODEL_DIRECTORY = "models"
# -------------------------------- Model names
CLASSIFICATION_MODEL_NAME = "classification_model.keras"
AUTOENCODER_MODEL_NAME = "autoencoder_model.keras"
CAPTIONNING_ENCODER_MODEL_NAME = "captionning_encoder_model.keras"
CAPTIONNING_DECODER_MODEL_NAME = "captionning_decoder_model.keras"
# -------------------------------- Model paths
CLASSIFICATION_MODEL_PATH = os.path.join(MODEL_DIRECTORY, CLASSIFICATION_MODEL_NAME)
AUTOENCODER_MODEL_PATH = os.path.join(MODEL_DIRECTORY, AUTOENCODER_MODEL_NAME)
CAPTIONNING_ENCODER_MODEL_PATH = os.path.join(MODEL_DIRECTORY, CAPTIONNING_ENCODER_MODEL_NAME)
CAPTIONNING_DECODER_MODEL_PATH = os.path.join(MODEL_DIRECTORY, CAPTIONNING_DECODER_MODEL_NAME)

## Partie 1 : Classification

### 1.0 - Mise au propre des folders & tri des fichiers

In [ ]:
def is_image(filename):
    try:
        with Image.open(filename) as img:
            img.verify()
        return True
    except (UnidentifiedImageError, OSError):
        return False
def move_non_images(directory):
    dump_directory = "dump"
    os.makedirs(dump_directory, exist_ok = True)
    
    for folder, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(folder, file)
            if not is_image(file_path):
                print(f"Déplacement de {file_path} dans le dossier dump/")
                dest_path = os.path.join(dump_directory, file)
                try:
                    shutil.move(file_path, dest_path)
                except:
                    print("Erreur lors du déplacement")
def is_valid_image(path):
    try:
        img_raw = tf.io.read_file(path)
        _ = tf.image.decode_image(img_raw, channels=3)
        return (path, True)
    except Exception:
        return (path, False)
def clean_corrupted_images(directory, extensions=("jpg", "jpeg", "png"), max_workers=8):
    image_paths = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(extensions):
                image_paths.append(os.path.join(root, file))

    print(f"Scan de {len(image_paths)} images dans {directory}")

    corrupted_count = 0
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(is_valid_image, path) for path in image_paths]
        for future in as_completed(futures):
            path, is_valid = future.result()
            if not is_valid:
                try:
                    os.remove(path)
                    corrupted_count += 1
                except Exception as e:
                    print(f"Erreur de suppression {path} : {e}")

    print(f"Vérification terminée : {corrupted_count} image(s) corrompue(s) supprimée(s).")

In [ ]:
move_non_images(DATASET_DIRECTORY)

clean_corrupted_images(DATASET_DIRECTORY)

# Remove Photos from previous iteration
for dir in [
    PHOTOS_DIRECTORY, 
    DENOISED_PHOTOS_DIRECTORY,
    ]:  
    # Remove the directory if it exists
    if os.path.exists(dir):
        shutil.rmtree(dir) 
    # Create the directory
    if not os.path.exists(dir):
        os.mkdir(dir)

### 1.1 - Import des données source

In [ ]:
dataset = keras.utils.image_dataset_from_directory(
    DATASET_DIRECTORY,
    batch_size = BATCH_S,
    image_size = (IMAGE_H, IMAGE_W),
    label_mode = None,
    seed = 42,
    validation_split = None,
    subset = None,
    shuffle = None,
)
filepaths = dataset.file_paths

### 1.2 - Classification des données

In [ ]:
# filter_model = load_model('../model_basic_cnn.keras')
classification_model = load_model(CLASSIFICATION_MODEL_PATH)

In [ ]:
predicts = classification_model.predict(dataset, verbose = 1)
y_pred = []
y_pred.extend(predicts.argmax(axis=1))

### 1.3 - Copie des photos dans un répertoire spécifique

In [ ]:
images_preds = list(zip(filepaths, y_pred))
photos_preds = list(filter(lambda x: x[1] == PHOTO_IDX,images_preds))

for filepath, prediction in images_preds:
    if prediction == PHOTO_IDX:
        filename = os.path.basename(filepath)
        # print(filename)
        dest_path = os.path.join(PHOTOS_DIRECTORY, filename)
        shutil.copy(filepath, dest_path)

## Partie 2 : Dénoising des images

### 2.0 - Import des données

In [ ]:
dataset = keras.utils.image_dataset_from_directory(
    directory = PHOTOS_DIRECTORY,
    batch_size = BATCH_S,
    image_size = (IMAGE_H, IMAGE_W),
    label_mode = None,
    seed = 42,
    validation_split = None,
    subset = None,
    shuffle = None,
)
filepaths = dataset.file_paths

In [ ]:
dataset = dataset.map(lambda x: tf.cast(x, tf.float32) / 255.0)
X = []
for batch in dataset:
    X.append(batch.numpy())
dataset = np.concatenate(X)

### 2.1 - Denoising des images sources

In [ ]:
# load model from file
autoencoder_model = load_model(AUTOENCODER_MODEL_PATH)

In [ ]:
denoised_images = autoencoder_model.predict(dataset, verbose = 1)

In [ ]:
plt.imshow(denoised_images[1])
plt.axis("off")
plt.show()

### 2.2 - Sauvegarde des images

In [ ]:
# save images from denoised_images
for i, image in enumerate(denoised_images):
    # Convert the image to uint8 format
    image = (image * 255).astype(np.uint8)
    # Create a PIL Image from the numpy array
    pil_image = Image.fromarray(image)
    # Save the image
    filename = os.path.basename(filepaths[i])
    dest_path = os.path.join(DENOISED_PHOTOS_DIRECTORY, filename)
    pil_image.save(dest_path)

## Partie 3 : Captioning des images

### 3.0 - Import des données

In [ ]:
dataset = keras.utils.image_dataset_from_directory(
    directory = DENOISED_PHOTOS_DIRECTORY,
    label_mode = None,
    batch_size = BATCH_S,
    image_size = (IMAGE_H, IMAGE_W),
    seed = 42
)

### 3.1 - Prétraitement des données

In [ ]:
dataset = dataset.map(lambda x: tf.cast(x, tf.float32) / 255.0)

for batch in dataset:
    X.append(batch.numpy())
    
dataset = np.concatenate(X)

### 3.2 - Définition de class utiles à l'import des modèles

In [ ]:
@tf.keras.utils.register_keras_serializable('CNN_Encoder')
class CNN_Encoder(tf.keras.Model):
    # Comme les images sont déjà prétraités par InceptionV3 est représenté sous forme compacte
    # L'encodeur CNN ne fera que transmettre ces caractéristiques à une couche dense
    def __init__(self, embedding_dim, **kwargs):
        super(CNN_Encoder, self).__init__(**kwargs)
        self.embedding_dim = embedding_dim
        # forme après fc == (batch_size, 64, embedding_dim)
        self.fc = tf.keras.layers.Dense(embedding_dim)
    def call(self, x):
        x = self.fc(x)
        x = tf.nn.relu(x)
        return x
    def get_config(self):
        config = super(CNN_Encoder, self).get_config()
        config.update({
            'embedding_dim': self.embedding_dim,
        })
        return config

In [4]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        attention_hidden_layer = tf.nn.tanh(
                self.W1(features) + self.W2(hidden_with_time_axis)
        )

        score = self.V(attention_hidden_layer)

        attention_weights = tf.nn.softmax(score, axis=1)

        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        return context_vector, attention_weights

In [8]:
@tf.keras.utils.register_keras_serializable("RNN_Decoder")
class RNN_Decoder(tf.keras.Model):
    def __init__(self, embedding_dim, units, vocab_size, use_lstm=False, **kwargs):
        super(RNN_Decoder, self).__init__(**kwargs)
        self.units = units
        self.vocab_size = vocab_size

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.use_lstm = use_lstm
        if not use_lstm:
            self.layer = tf.keras.layers.GRU(
                self.units,
                return_sequences=True,
                return_state=True,
                #recurrent_initializer='glorot_uniform',
                
                activation='tanh',
                recurrent_activation='sigmoid',
                use_bias=True,
                kernel_initializer='glorot_uniform',
                recurrent_initializer='orthogonal',
                bias_initializer='zeros',
                unroll=True,
            )

        else:
            self.layer = tf.keras.layers.LSTM(
                self.units,
                return_sequences=True,
                return_state=True,
                #recurrent_initializer='glorot_uniform',
                
                activation='tanh',
                recurrent_activation='sigmoid',
                use_bias=True,
                kernel_initializer='glorot_uniform',
                recurrent_initializer='orthogonal',
                bias_initializer='zeros',
                unroll=True,
            )

        self.fc1 = tf.keras.layers.Dense(self.units)

        self.fc2 = tf.keras.layers.Dense(self.vocab_size)

        self.attention = BahdanauAttention(self.units)

    def call(self, x, features, hidden):
        context_vector, attention_weights = self.attention(features, hidden)
        x = self.embedding(x) 
        context_vector = tf.expand_dims(context_vector, 1)  
        x = tf.concat([context_vector, x], axis=-1)  
        if not self.use_lstm:
            output, state = self.layer(x)
        else:
            output, state, a = self.layer(x)  
        y = self.fc1(output)
        y = tf.reshape(y, (-1, y.shape[2]))
        y = self.fc2(y)
        return y, state, attention_weights

    def reset_state(self, batch_size):
       return tf.zeros((batch_size, self.units))
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "embedding_dim": self.embedding,
            "units": self.units,
            "vocab_size": self.vocab_size,
            "use_lstm": self.use_lstm
        })
        return config
    
    @classmethod
    def from_config(cls, config):
        return cls(
            embedding_dim=config["embedding_dim"],
            units=config["units"],
            vocab_size=config["vocab_size"],
            use_lstm=config.get("use_lstm", True)
        )

### 3.3 - Import des modèles

In [17]:
encoder_model = load_model(CAPTIONNING_ENCODER_MODEL_PATH)

In [16]:
decoder_model = load_model(CAPTIONNING_DECODER_MODEL_PATH)

c:\Users\Lucien\Projects\data-science\.venv\Lib\site-packages\keras\src\saving\serialization_lib.py:730: UserWarning: Model 'rnn__decoder_4' had a build config, but the model cannot be built automatically in `build_from_config(config)`. You should implement `def build_from_config(self, config)`, and you might also want to implement the method  that generates the config at saving time, `def get_build_config(self)`. The method `build_from_config()` is meant to create the state of the model (i.e. its variables) upon deserialization.
  instance.build_from_config(build_config)


### 3.4 - Captionning des images

In [ ]:
encoder_model.